In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/dinnerFoodTypes.zip", "dinnerFoodTypes/")

In [4]:
import psutil
import humanize
import os
from IPython.display import display_html

 

import numpy as np  
import pandas as pd  

 

import os
print(os.listdir("/content/dinnerFoodTypes/dinnerFoodTypes"))
dataDirectory= "/content/dinnerFoodTypes/dinnerFoodTypes"
print(os.listdir(dataDirectory))

['test', 'train', 'valid']
['test', 'train', 'valid']


In [5]:
train_path = dataDirectory+'/train'
test_path  = dataDirectory+'/valid'
print(os.listdir(train_path))
print(os.listdir(test_path))

['1', '0', '3', '2']
['1', '0', '3', '2']


In [6]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import itertools
import matplotlib.pyplot as plt
import time
import pandas as pd
%matplotlib inline
import keras.applications.xception as xception


In [7]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest',
    validation_split=0.2)

In [8]:
selectedClasses = ['0', '1','2','3']

In [9]:
train_path = '/content/dinnerFoodTypes/dinnerFoodTypes/train'
test_path  = '/content/dinnerFoodTypes/dinnerFoodTypes/test'

In [10]:
batchSize=32


train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=batchSize,
    classes=selectedClasses,
    subset='validation') # set as validation data

test_generator = ImageDataGenerator().flow_from_directory(
    test_path,
    target_size=(224,224),
    classes=selectedClasses,
    shuffle= False,
    batch_size = batchSize)# set as test data

Found 1256 images belonging to 4 classes.
Found 312 images belonging to 4 classes.
Found 150 images belonging to 4 classes.


In [11]:
print ("In train_generator ")
for cls in range(len (train_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(train_generator.classes).count(cls))
print ("")

print ("In validation_generator ")
for cls in range(len (validation_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(validation_generator.classes).count(cls))
print ("")

print ("In test_generator ")
for cls in range(len (test_generator.class_indices)):
    print(selectedClasses[cls],":\t",list(test_generator.classes).count(cls))

In train_generator 
0 :	 474
1 :	 410
2 :	 100
3 :	 272

In validation_generator 
0 :	 118
1 :	 102
2 :	 25
3 :	 67

In test_generator 
0 :	 72
1 :	 67
2 :	 9
3 :	 2


In [12]:
train_generator.reset()
imgs, labels = train_generator.next()

#print(labels)

labelNames=[]
labelIndices=[np.where(r==1)[0][0] for r in labels]
#print(labelIndices)

for ind in labelIndices:
    for labelName,labelIndex in train_generator.class_indices.items():
        if labelIndex == ind:
            #print (labelName)
            labelNames.append(labelName)

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
num_classes=4
IMAGE_SHAPE = [224, 224]  
batch_size=32
vgg = VGG16(input_shape = (224,224,3), weights = 'imagenet', include_top = False)  

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(128, activation = 'relu')(x)    
x = Dense(64, activation = 'relu')(x)
x = Dense(num_classes, activation = 'softmax')(x)   

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
from keras.preprocessing.image import ImageDataGenerator
trdata = ImageDataGenerator()
train_data_gen = trdata.flow_from_directory(directory="/content/dinnerFoodTypes/dinnerFoodTypes/train",target_size=(224,224),shuffle=False, class_mode='categorical')
tsdata = ImageDataGenerator()
test_data_gen = tsdata.flow_from_directory(directory="/content/dinnerFoodTypes/dinnerFoodTypes/test", target_size=(224,224),shuffle=False, class_mode='categorical')

Found 1568 images belonging to 4 classes.
Found 150 images belonging to 4 classes.


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
epochs = 50
checkpoint = ModelCheckpoint(filepath='/content/finalvgg16model.h5', verbose=1, save_best_only=True)
training_steps_per_epoch = np.ceil(train_data_gen.samples / batch_size)
validation_steps_per_epoch = np.ceil(test_data_gen.samples / batch_size)

model.fit_generator(train_data_gen, steps_per_epoch=training_steps_per_epoch, validation_data=test_data_gen, validation_steps=validation_steps_per_epoch,
                        epochs=epochs, verbose=1)
print('Training Completed!')

<ipython-input-15-a33cba508e89>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data_gen, steps_per_epoch=training_steps_per_epoch, validation_data=test_data_gen, validation_steps=validation_steps_per_epoch,


Epoch 1/50
49/49 [==============================] - 30s 424ms/step - loss: 21.6780 - accuracy: 0.6448 - val_loss: 3.7162 - val_accuracy: 0.9000
Epoch 2/50
49/49 [==============================] - 16s 321ms/step - loss: 1.7492 - accuracy: 0.8807 - val_loss: 4.4011e-04 - val_accuracy: 1.0000
Epoch 3/50
49/49 [==============================] - 15s 302ms/step - loss: 0.2900 - accuracy: 0.9726 - val_loss: 2.3565e-05 - val_accuracy: 1.0000
Epoch 4/50
49/49 [==============================] - 16s 316ms/step - loss: 0.1075 - accuracy: 0.9923 - val_loss: 0.0126 - val_accuracy: 0.9867
Epoch 5/50
49/49 [==============================] - 15s 313ms/step - loss: 0.1207 - accuracy: 0.9943 - val_loss: 4.3869e-07 - val_accuracy: 1.0000
Epoch 6/50
49/49 [==============================] - 16s 321ms/step - loss: 0.0649 - accuracy: 0.9911 - val_loss: 8.8641e-06 - val_accuracy: 1.0000
Epoch 7/50
49/49 [==============================] - 16s 318ms/step - loss: 0.0567 - accuracy: 0.9949 - val_loss: 2.1063e-05 -

In [16]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_data_gen.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

5/5 [==============================] - 2s 405ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        72
           1       1.00      1.00      1.00        67
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00         2

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150



In [17]:
keras_file="dinnerfoodtype.h5"
tf.keras.models.save_model(model,keras_file)

<ipython-input-17-e4483b396b19>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model,keras_file)


In [18]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/dinnerfoodtype.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("dinnerfoodtype.tflite", "wb").write(tflite_model)

71748116

In [19]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
img_path = '/content/mytest.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds=model.predict(x)
# create a list containing the class labels
class_labels = ['rice','stew','beans','pasta']

# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)

# print the label of the class with maximum score
print(class_labels[pred[0]])

1/1 [==============================] - 2s 2s/step
rice
